In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

df=pd.read_csv("fil_data_movies_lessthan10k_removed.csv")

In [2]:
#added column for decade
df['decade'] = (pd.DatetimeIndex(df['release_date']).year // 10) * 10

In [3]:
s = df['genres'].str.split(', ').explode()
encoder = OneHotEncoder()
encoded = encoder.fit_transform(s.values[:, None])
one_hot_df = pd.DataFrame(encoded.toarray(), columns=np.ravel(encoder.categories_), dtype='int') \
                .groupby(s.index) \
                .sum()

final = pd.concat([df, one_hot_df], axis=1)

In [4]:
#top 10 movie genres of all time
genre_counts = final.iloc[:, 23:].sum()
genre_percents = (genre_counts / len(df)) * 100
genre_percents=genre_percents.sort_values(ascending=False)
genre_percents[:10]

Drama              45.753830
Comedy             36.764311
Thriller           25.718893
Action             25.463585
Adventure          18.341844
Romance            18.153722
Crime              15.990325
Horror             11.824778
Science Fiction    11.421661
Family             10.467616
dtype: float64

In [5]:
#average runtime per year

runtime = final.groupby('decade')['runtime']
#runtime.mean() #filmid tegelt ei ole ajaga pikemaks läinud
#runtime.count() #filme kümnenditest 1910,1920 on liiga vähe et mingit arvamust luua

In [6]:
#vote_avg per year

votes = final.groupby('decade')['vote_average']
#votes.mean() #vanemad filmid veits kõrgema ratinguga, samas kui sa hakkad 80a vana filmi vaatama ss see pole mingi random

In [7]:
#top10 genres
genres = ("Drama","Comedy","Thriller","Action","Adventure","Romance","Crime","Horror","Science Fiction","Family")
total_movies_by_decade = final.groupby('decade')['id'].count()


genres_by_decade_count = final.groupby('decade')[['id']].count()
for genre in genres:
    genre_movie_by_decade = final.groupby('decade')[genre].sum()
    genre_percent_by_decade = (genre_movie_by_decade / total_movies_by_decade) * 100
    genres_by_decade_count[genre+'%'] = genre_percent_by_decade

genres_by_decade_count

,id,Drama%,Comedy%,Thriller%,Action%,Adventure%,Romance%,Crime%,Horror%,Science Fiction%,Family%
decade,,,,,,,,,,,
1910,2,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1920,15,73.333333,40.000000,20.000000,20.000000,20.000000,46.666667,6.666667,0.000000,6.666667,0.000000
1930,48,54.166667,52.083333,0.000000,4.166667,14.583333,45.833333,12.500000,10.416667,6.250000,6.250000
1940,81,49.382716,19.753086,27.160494,3.703704,11.111111,37.037037,18.518519,4.938272,0.000000,9.876543
1950,111,53.153153,22.522523,15.315315,5.405405,12.612613,40.540541,17.117117,9.909910,14.414414,7.207207
1960,153,52.941176,28.758170,15.032680,11.111111,20.261438,25.490196,9.803922,6.535948,3.921569,5.882353
1970,248,45.564516,31.854839,27.016129,26.612903,20.564516,9.677419,18.548387,10.887097,16.532258,5.241935
1980,699,34.763948,40.343348,21.316166,24.463519,19.456366,16.738197,14.878398,18.741059,16.595136,7.868383
1990,1175,46.468085,41.446809,26.553191,24.595745,15.574468,19.063830,20.000000,10.468085,12.765957,11.829787


In [8]:
###todo:

#otsida seost kui kümnendil palju ühe žanri filme == selle žanri filmid teevad head kassat? 

